# Motivation
论文原文：[《TabNet: Attentive Interpretable Tabular Learning》](https://arxiv.org/abs/1908.07442)

参考学习地址：
* [数据挖掘竞赛利器——TabNet模型浅析](https://zhuanlan.zhihu.com/p/152211918)
* [TabNet-为表格数据而生](https://zhuanlan.zhihu.com/p/271362118)
* [重新梳理一下tabnet](https://zhuanlan.zhihu.com/p/506945010)

代码库：https://github.com/dreamquark-ai/tabnet

TabNet，它在保留DNN的end-to-end和representation learning特点的基础上，还拥有了树模型的可解释性和稀疏特征选择的优点。
* 减轻特征工程需要，无需预处理直接使用原数据，基于梯度下降优化训练，端到端模型允许表示学习，支持从流数据学习
* 每个决策步中，TabNet使用顺序注意力来选择哪些特征进行推理，使可解释性和更好学习成为可能（局部：可视化特征的重要性及其组合方式，全局：量化每个特征对训练模型的贡献）

<img style="display: block; margin: 0 auto;" src="../../../assets/images/tabnet-0.png" width = "1000" height = "400" alt="TabNet-0" align=center />

# 用DNN构建决策树
<img style="display: block; margin: 0 auto;" src="../../../assets/images/tabnet-1.png" width = "1000" height = "400" alt="TabNet-1" align=center />

传统决策树如右图坐标表示，输入两个特征$x_1$和$x_2$，分别以$a$和$d$为阈值进行划分。使用DNN，则输入为特征向量$[x_1,x_2]$，分别通过两个Mask层筛选，然后通过weight和bias被专门设定过的全连接层，将两个FC的输出通过ReLU激活函数后相加，最后通过Softmax激活输出。与决策树流程对比，可以发现DNN的每一层对应决策树的相应步骤：
* Mask层对应决策树中的**特征选择**：DT常用的方式是提取统计信息增益最多的全局特征，RF进一步通过bagging减少方差，XGB/LGBM通过boosting减少偏差
* FC层+ReLU对应**阈值判断**：构建一个连续的多步骤体系结构，其中每个步骤基于所选特征对决策的一部分作出贡献；通过对所选特征进行非线性处理，提高学习能力；通过更高的维度和更多的步骤来模拟ensemble

以$x_1$为例，经过FC后获取向量$[C_1x_1-C_1a,-C_1x_1+C_1a,-1,-1]$，ReLU激活后，有
$$
\begin{align}
output = 
\begin{cases}
    [C_1x_1-C_1a, 0, 0, 0], & \text{if } x_1 > a \\
    [0,-C_1x_1+C_1a, 0, 0] & \text{if } x_1 < a
\end{cases}
\end{align}
$$
所以输出的向量里面只有一个是正值，其余全为0，也对应了决策树的条件判断。而DNN本质为<font color="red">加性模型（Additive Model）</font>，一个Mask+FC+ReLU组合对应一个基本决策树，上例中对应两棵树分别挑选$x_1$和$x_2$作为划分特征，最终输出加和，向量可以理解成一个权重向量，它的每一维代表**某个条件判断的对最终决策影响的权重**。与决策树不同，例如上例中假设条件为$x_1>a, x_2>d$，则输出为$[C_1x_1-C_1a,0,C_2x_2-C_2d,0]$的softmax值，对应位置的权重为条件对应重要性权重，而决策树中如果条件成立其权重恒为1。因此，TabNet其实是一个*“软”版本的决策树加性模型*。

# 模型架构
<img style="display: block; margin: 0 auto;" src="../../../assets/images/tabnet-2.png" width = "800" height = "600" alt="TabNet-2" align=center />

模型与实例中基本一致，可以看成是一个有很多step的加性模型，模型输入是维度为$B\times D$的特征，其中$B$为batch size，$D$为feature的维度，模型输出是一个向量或一个数。
* input部分：numeric不变，category做embedding，然后两者concat后输入BN；
* BN层：也即Batch Normalization层，原文中使用的是ghost batch norm来增强BN的鲁棒性，推荐的batch_size是全量样本的1%-10%。但通常千万级的样本对应10W，普通GPU有压力，而如果batch_size太小，效果会差很多；
* Feature Transformer层：作用类似之前的FC层（如果用FC替换GLU，其实就是两个resnet block叠加），用于特征计算，结构相对复杂。
  * 其中GLU（Gated Linear Unit），是在原始FC层基础上增加门控，计算公式为$h(X)=(W*X+b)\bigotimes \sigma(V*X+c)$，Tips：每一个GLU的input和output的size都是一样的
  * 前半部分层参数共享，也即在所有step上共同训练；后半部分各step独立分开训练。原因是对于每一个step，输入是同样的features（Mask仅屏蔽部分features，并不改变其他features），因此用**共享层计算特征的共性部分**，之后通过不同层做每一个step的**特性部分**
  * 层中使用了残差连接，乘$\sqrt(0.5)$用于放缩反向传播的梯度，避免太大的梯度使得weights的方差大，是为了保证网络的稳定性
* Split层：将Feature Transformer输出的向量切成两部分，$[d[i],a[i]]=f_i(M[i]·f)$，其中$d[i]$用于计算模型的最终输出（超参数$n_d$控制，n features for decision），$a[i]$则用来计算下一个step的Mask层（超参数$n_a$控制，n features for attention）
* Attentive Transformer层：根据上一个step的结果（对应$a[i]$部分），计算出当前step的Mask层。$M[i]=Sparsemax(P[i-1]·h_i(a[i-1]))\in \R^{B\times D}$
  * 其中$a[i-1]$是上一个step中split层划分出来的
  * $h_i(·)$代表FC+BN层
  * $P[i]=\prod_{j=1}^i(\gamma-M[j])$是Prior scales项，用来表示某一个feature在之前的step中的**运用程度**，按照直觉，如果一个feature在之前的step中用了很多次，则不应再被选中。Prior scales项用来减少这类feature的权重占比，如果另$\gamma=1$，那么每个feature只能被使用一次，当$\gamma$增加时，这个约束会放宽
  * Sparsemax层可以理解为Softmax的稀疏化版本（**可以看作是relu版的softmax**）。不同于Softmax的平滑变换，Sparsemax通过直接将向量投影到一个simplex来实现稀疏化，$Sparsemax(z)=argmin_{p\in \bigtriangleup}||p-z||$。因为Mask矩阵是$B\times D$维的，根据Sparsemax性质$\sum_{j=1}^{D}M[i]_{b,j}=1$，因此$M[i]$可以理解为模型在当前step上，对batch样本的**注意力权重分配**，Tips：对于不同样本，Attentive Transformer层输出的注意力权重也不同，这个特点在论文中被叫做**instance-wise**
$$
\begin{align}
sparsemax_1(z) = 
\begin{cases}
    1, & \text{if } t > 1 \\
    (t+1)/2, & \text{if } -1 \le t \le 1 \\
    0, & \text{if } t \le -1 \\
\end{cases}
\end{align}
$$
* 正则项：目标为了增强模型对feature稀疏选择的能力，$L_{\text {sparse}}=\sum_{i=1}^{N_{s t e p s}} \sum_{b=1}^{B} \sum_{j=1}^{D} \frac{-M_{b, j}[\mathrm{i}]}{N_{s t e p s} \cdot B} \log \left(M_{b, j}[i]+\epsilon\right)$。本质是计算了一个平均的熵，目的是希望$M[i]$分别尽量趋近于0或1，而考虑到$\sum_{j-1}^{D}M[i]_{b,j}=1$，则$L_{sparse}$反映的就是$M[i]$的**稀疏程度**，$L_{sparse}$越小，$M[i]$越稀疏
* Feature attribute输出：刻画feature的全局重要性，模型先对模型一个step的输出向量求和，所得标量反映这个step对最终结果的重要性，它乘以这个step的Mask矩阵就反映了这个step中每个feature的重要性，将所有step结果相加也即得到了feature的全局重要性

总结而言，TabNet采用了**顺序多步（sequential multi-step）框架，构造了一种类似于加性模型的神经网络，其中关键的是Attentive Transformer层和Feature Transformer层：
* 特征选择：Attentive Transformer层根据上一个step结果得到当前step的Mask矩阵，并尽量使Mask矩阵**稀疏且不重复**。Tips：不同样本的Mask向量可以不同，也就是说TabNet可以让**不同的样本选择不同的特征（instance-wise）**，而这个特点是树模型所不具备的，对于XGBoost这类加性模型，一个step就是一棵树，而这棵决策树用到的特征是在所有样本上挑选出来的（例如通过计算信息增益），它没有办法做到instance-wise。
* 特征计算：Feature Transformer层实现了对于当前step步所选取特征的计算处理。决策树中对于给定的特征，其构建是根据单个特征的大小关系组合，DNN中通过一个FC来模仿这个流程，但FC仅构造简单的线性关系。TabNet通过更复杂的Feature Transformer来特征计算，学习模式更加复杂

# 自监督学习
<img style="display: block; margin: 0 auto;" src="../../../assets/images/tabnet-3.png" width = "800" height = "500" alt="TabNet-3" align=center />

DNN的好处之一是可以表征学习，TabNet使用自监督学习方法，通过Encoder-Decoder框架来获取表格数据的representation，有助于分类和回归任务。简单而言，是假设同一样本的不同特征之间有关联，因此自监督学习是先人为mask一些feature，然后通过encoder-decoder模型来对mask的feature预测。我们认为通过这样的方式训练得到的encoder模型，可以有效表征样本的feature（可理解为对数据进行了编码或压缩），这时再将encoder模型用于回归或分类任务就可事半功倍。Decoder模型中Encoded representation对应Encoder中**没有经过FC层的加和向量**。Decoder同样利用了Feature transformer层，只不过这次的目的是将representation向量重构为feature，然后类似地经过若干个step的加和，得到最后的重构feature。

假设一开始对feature做mask的矩阵是$S\in \{0, 1\}^{B\times D}$，特征数据是$f$，则encoder输入为$(1-S)\cdot f$，若decoder的输出是$\hat{f}$，则自监督学习就是减少真实值$S\cdot f$和重构值$S\cdot \hat{f}$之间的差别。考虑不同feature量级不一定相同，因此采用正则化的MSE作为loss，形式为：
$$
\begin{align}
\sum_{b=1}^{B} \sum_{j=1}^{D}\left|\left(\hat{f}_{b, j}-f_{b, j}\right) \cdot S_{b, j} / \sqrt{\sum_{b=1}^{B}\left(f_{b, j}-1 / B \sum_{b=1}^{B} f_{b, j}\right)^{2}}\right|^{2}
\end{align}
$$

另外，为了让模型学到的是整个feature数据的表征方法，而不仅仅是某些feature，在自监督学习的训练过程中，每一轮都会对矩阵 $S$ **重新采样**，以此来保证encoder模型的整体表征能力。总结一下，自监督学习可以让encoder模型学到feature数据的representation，通过representation再对任务进行分类和回归就会变得更加容易了。类似于NLP中的pre-training和fine-tuning过程，自监督学习可以很好地利用到没有label的数据，从而使得模型在有label的样本比较少时也能取得不错的效果，并且还能加快模型的收敛速度。